In [1]:
import wobble
import numpy as np
import matplotlib.pyplot as plt

/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = wobble.Data('51peg_e2ds.hdf5', filepath='data/', orders=[56])
results = wobble.Results(data)

In [3]:
model = wobble.Model(data, results, 0)

In [4]:
model.add_star('star')
model.add_telluric('tellurics', rvs_fixed=True, variable_bases=3)

In [5]:
model.setup()

In [6]:
for c in model.components:
    results.add_component(c)

In [ ]:
model.optimize()

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70


In [ ]:
plt.plot(results.star_template_xs[0], results.star_template_ys[0])